In [2]:
def create_database_in_mysql():
    import mysql.connector
    import json
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
        )
    
    # Execute a SQL query to create the database
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS `creditcard_capstone`")
        conn.commit()
        print("Database created successfully")
    except Exception as e:
        print(f"Error creating database: {e}")
    finally:
    # Close the Spark session
        cursor.close()
        conn.close()

In [1]:
def create_table_in_mysql():
    from pyspark.sql import SparkSession
    try:
    # Create a Spark session
        spark = SparkSession.builder.appName("CreateTableInMySQL").config("spark.jars", r"E:\soft\Spark\spark-3.5.0-bin-hadoop3\jars\mysql-connector-j-8.3.0.jar").getOrCreate()

        # Define the JDBC URL for the MySQL database
        jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"

        table_schema1 = """
            ID INT AUTO_INCREMENT PRIMARY KEY,
            FIRST_NAME VARCHAR(255),
            MIDDLE_NAME VARCHAR(255),
            LAST_NAME VARCHAR(255),
            SSN INT,
            CREDIT_CARD_NO VARCHAR(255),
            APT_NO VARCHAR(255),
            STREET_NAME VARCHAR(255),
            CUST_CITY VARCHAR(255),
            CUST_STATE VARCHAR(255),
            CUST_COUNTRY VARCHAR(255),
            CUST_ZIP VARCHAR(255),
            CUST_PHONE INT,
            CUST_EMAIL VARCHAR(255),
            LAST_UPDATED DATETIME,
            INDEX inx_credit_card_no (CREDIT_CARD_NO)
        """
        table_schema2 = """
            BRANCH_CODE INT PRIMARY KEY ,
            BRANCH_NAME VARCHAR(255),
            BRANCH_STREET VARCHAR(255),
            BRANCH_CITY VARCHAR(255),
            BRANCH_STATE VARCHAR(255),
            BRANCH_ZIP INT,
            BRANCH_PHONE VARCHAR(255),
            LAST_UPDATED DATETIME
        """
        table_schema3 = """
            TRANSACTION_ID INT PRIMARY KEY,
            DAY INT,
            MONTH INT,
            YEAR INT,
            CREDIT_CARD_NO VARCHAR(255),
            CUST_SSN INT,
            BRANCH_CODE INT,
            TRANSACTION_TYPE VARCHAR(255),
            TRANSACTION_VALUE INT,
            CONSTRAINT `branch_fk` FOREIGN KEY (`BRANCH_CODE`) REFERENCES `CDW_SAPP_BRANCH` (`BRANCH_CODE`),
            CONSTRAINT `credit_card_fk` FOREIGN KEY (CREDIT_CARD_NO) REFERENCES `CDW_SAPP_CUSTOMER`(CREDIT_CARD_NO)
        """
        df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer.json")
        df2 = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")
        df3 = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")

    # Execute the SQL query to create the table in the MySQL database
    
        # Write the DataFrame to the MySQL table with the specified schema and index
        df.write.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "CDW_SAPP_CUSTOMER") \
        .option("user", "root") \
        .option("password", "password") \
        .mode("overwrite") \
        .save()
        
        df2.write.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "CDW_SAPP_BRANCH") \
        .option("user", "root") \
        .option("password", "password") \
        .mode("overwrite") \
        .save()
        
        df3.write.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
        .option("user", "root") \
        .option("password", "password") \
        .mode("overwrite") \
        .save()
    except Exception as e:
        print(f"Error creating table: {e}")
        
    finally:
    # Close the Spark session
        spark.stop()

In [6]:
def display_customer_table():
    # pyspark code to read customer json file
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import initcap, lower,concat,lit,regexp_replace
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("Read_Customer_JSON").getOrCreate()

        # Specify the path to the JSON file
        json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_custmer.json"

        # Read the JSON file into a DataFrame
        df = spark.read.json(json_file_path)
        df = df.withColumn("CUST_PHONE", concat(lit("337"), df["CUST_PHONE"]))

        formated_df = df.withColumn("ADDRESS", concat(df["APT_NO"], lit(", "), df["STREET_NAME"])) \
                        .withColumn("CUST_PHONE", regexp_replace(df["CUST_PHONE"].cast("string"), "(\\d{3})(\\d{3})(\\d{4})", "($1)$2-$3")) \
                        .withColumn("FIRST_NAME", initcap("FIRST_NAME")) \
                        .withColumn("MIDDLE_NAME", lower("MIDDLE_NAME")) \
                        .withColumn("LAST_NAME", initcap("LAST_NAME"))

        # Show the DataFrame
        formated_df.select("FIRST_NAME", "MIDDLE_NAME", "LAST_NAME", "SSN", "CREDIT_CARD_NO", "ADDRESS", "CUST_CITY", "CUST_STATE", "CUST_COUNTRY", "CUST_ZIP", "CUST_PHONE", "CUST_EMAIL", "LAST_UPDATED").show(formated_df.count(),truncate=False)
    except Exception as e:
        print(e)
    finally:
        # Stop the SparkSession
        spark.stop()
        
display_customer_table()

+----------+-----------+-------------+---------+----------------+------------------------+-----------------+----------+-------------+--------+-------------+--------------------------+-----------------------------+
|FIRST_NAME|MIDDLE_NAME|LAST_NAME    |SSN      |CREDIT_CARD_NO  |ADDRESS                 |CUST_CITY        |CUST_STATE|CUST_COUNTRY |CUST_ZIP|CUST_PHONE   |CUST_EMAIL                |LAST_UPDATED                 |
+----------+-----------+-------------+---------+----------------+------------------------+-----------------+----------+-------------+--------+-------------+--------------------------+-----------------------------+
|Alec      |wm         |Hooper       |123456100|4210653310061055|656, Main Street North  |Natchez          |MS        |United States|39120   |(337)123-7818|AHooper@example.com       |2018-04-21T12:49:02.000-04:00|
|Etta      |brendan    |Holman       |123453023|4210653310102868|829, Redwood Drive      |Wethersfield     |CT        |United States|06109   |(3

In [6]:
def display_branch_table():
    # pyspark to read branch json file
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import initcap, lower,concat,lit,regexp_replace
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("Read_branch_JSON").getOrCreate()

        # Specify the path to the JSON file
        json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json"

        # Read the JSON file into a DataFrame
        df = spark.read.json(json_file_path)
        formated_df = df.withColumn("BRANCH_PHONE", regexp_replace(df["BRANCH_PHONE"].cast("string"), "(\\d{3})(\\d{3})(\\d{4})", "($1)$2-$3"))
     
    # Show the DataFrame
        formated_df.select("BRANCH_CODE","BRANCH_NAME","BRANCH_STREET","BRANCH_CITY","BRANCH_STATE","BRANCH_ZIP","BRANCH_PHONE","LAST_UPDATED").show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [7]:
def display_credit_table():
    # pyspark to read credit json file
    from pyspark.sql import SparkSession
    
    # Create a SparkSession
    spark = SparkSession.builder.appName("Read_credit_JSON").getOrCreate()

    # Specify the path to the JSON file
    json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json"

    # Read the JSON file into a DataFrame
    df = spark.read.json(json_file_path)

    try: 
    # Show the DataFrame
        df.select("TRANSACTION_ID","DAY","MONTH","YEAR","CREDIT_CARD_NO","CUST_SSN","BRANCH_CODE","TRANSACTION_TYPE","TRANSACTION_VALUE").show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    # Stop the SparkSession
    finally:
        spark.stop()


In [11]:
# Prompt the user for a zip code, provide contextual cues for valid input, and verify it is in the correct format.
def prompt_user_zipcode():
    try:
        while True:
            zipcode = input("Enter your zipcode (Enter 5-digit number): ")
            if len(zipcode) == 5 and zipcode.isdigit():
                return int(zipcode)
            else:
                print("Invalid zipcode. Please enter a 5-digit number.")
                
    except Exception as e:
        print(e)

def prompt_user_month():
    try:
        while True:
            month_input = input("Enter your month (Enter number 1 - 12): ")
            if month_input.isdigit():
                if 1 <= int(month_input) <= 12:
                    return int(month_input)
                else:
                    print("Invalid month. Please enter a number between 1 and 12.")
            else:
                print("Invalid Input. Please enter a number between 1 and 12.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

def prompt_user_year():
    try:
        while True:
            year_input = input("Enter your year (Enter 4-digit number): ")
            if len(year_input) == 4 and year_input.isdigit():
                return int(year_input)
            else:
                print("Invalid year. Please enter a 4-digit number.")
    except Exception as e:
        print(e)


In [4]:
def query_transaction_data_spark(zipcode, month, year):
    from pyspark.sql import SparkSession
    import json
    # Create a Spark session
    try:
        spark = SparkSession.builder.appName("QueryTransactionData").getOrCreate()

        # Load JSON data into DataFrames
        branch_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")
        transaction_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")

        # Load data from MySQL database using JDBC
        jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"
        properties = {
            "user": "root",
            "password": "password",
            "driver": "com.mysql.cj.jdbc.Driver"
        }

        # Query the data using Spark SQL
        branch_df.createOrReplaceTempView("branch")
        transaction_df.createOrReplaceTempView("transaction")
        
        query = f"""
            SELECT branch.BRANCH_CODE, branch.BRANCH_ZIP, transaction.DAY, transaction.MONTH, transaction.YEAR, branch.BRANCH_NAME, branch.BRANCH_STREET, branch.BRANCH_CITY, branch.BRANCH_STATE, branch.BRANCH_PHONE, transaction.CREDIT_CARD_NO, transaction.CUST_SSN, transaction.TRANSACTION_ID, transaction.TRANSACTION_TYPE, transaction.TRANSACTION_VALUE, branch.LAST_UPDATED
            FROM branch 
            JOIN transaction 
            Using (BRANCH_CODE) 
            WHERE branch.BRANCH_ZIP = {zipcode} 
            AND transaction.MONTH = {month} 
            AND transaction.YEAR = {year}
        """
        result_df = spark.sql(query)

        # Show the result DataFrame
        result_df.show()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:  
        # Stop the Spark session
        spark.stop()

In [11]:
def sort_transaction_by_day():
    # pyspark to read credit json file
    from pyspark.sql import SparkSession
    # Create a SparkSession
    spark = SparkSession.builder.appName("Read_credit_JSON").getOrCreate()

    # Specify the path to the JSON file
    json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json"

    # Read the JSON file into a DataFrame
    df = spark.read.json(json_file_path)

    try: 
    # Show the DataFrame
        df.select("TRANSACTION_ID","DAY","MONTH","YEAR","CREDIT_CARD_NO","CUST_SSN","BRANCH_CODE","TRANSACTION_TYPE","TRANSACTION_VALUE").orderBy("DAY", ascending=False).show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [20]:
def transaction_number_value(type):
    from pyspark.sql import SparkSession
    # Create a SparkSession
    spark = SparkSession.builder.appName("trasaction_JSON").getOrCreate()
    # Specify the path to the JSON file
    json_file_path = r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json"
    # Read the JSON file into a DataFrame
    df = spark.read.json(json_file_path)

    try: 
    # Show the DataFrame
        df.createOrReplaceTempView("transaction")
        spark.sql(f"""SELECT TRANSACTION_TYPE, count(TRANSACTION_TYPE) as Number, round(Sum(TRANSACTION_VALUE),2) as Total 
                    FROM transaction 
                    Group By TRANSACTION_TYPE 
                    HAVING TRANSACTION_TYPE = '{type}'""").show(df.count(),truncate=False)
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()


In [22]:
def prompt_user_type():
    try:
        while True:
            type = input("Enter Transaction Type (Healthcare, Automotive, etc.): ")
            if type.isalpha():
                return type
            else:
                print("Invalid input. Please enter string only.")
                
    except Exception as e:
        print(e)

In [5]:
def transaction_branch_number_value_state(state):
    from pyspark.sql import SparkSession
    try:
    # Create a SparkSession
        spark = SparkSession.builder.appName("trasaction_JSON").getOrCreate()
        # Read the JSON file into a DataFrame
        df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_credit.json")
        branch_df = spark.read.json(r"D:\CAP 405 Data Analytics - Capstone Project\cdw_sapp_branch.json")

        
        # Show the DataFrame
        df.createOrReplaceTempView("transaction")
        branch_df.createOrReplaceTempView("branch")
        
        query = f"""
            SELECT branch.BRANCH_STATE, count(TRANSACTION_TYPE) as Number, round(Sum(TRANSACTION_VALUE),2) as Total
            FROM branch 
            JOIN transaction 
            Using (BRANCH_CODE)
            WHERE branch.BRANCH_STATE = '{state}'
            Group By branch.BRANCH_STATE
        """
        
        result_df = spark.sql(query)
        # Show the result DataFrame
        result_df.show()
        
    except Exception as e: 
        print(e)
    finally:
    # Stop the SparkSession
        spark.stop()

In [9]:
def prompt_user_state():
    try:
        while True:
            state = input("Enter State (LA, CA, etc.): ")
            if state.isalpha():
                if len(state) == 2:
                    return state.upper()
                else:
                    print("Invalid input. Please enter 2-letters for the state only.")
            else:
                print("Invalid input. Please enter string only.")
                
    except Exception as e:
        print(e)

Invalid input. Please enter 2-letters for the state only.


'IL'